In [89]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

import time

In [1]:
import math
import pandas as pd
from word2number import w2n

## Get all stimuli

In [124]:
self_report_prompt = "How are stripes and spots changing the segments?"

construct = [
  "Observation: a magic egg with one stripe, touching three segments\nResulting segments: three\n",
  "Observation: a magic egg with two stripes, touching two segments\nResulting segments: four\n",
  "Observation: a magic egg with three stripes, touching one segment\nResulting segments: three\n",
]

decon = [
  "Observation: a magic egg with one stripe and three spots, touching three segments\nResulting segments: none\n",
  "Observation: a magic egg with two stripes and two spots, touching two segments\nResulting segments: two\n",
  "Observation: a magic egg with three stripes and one spot, touching one segment\nResulting segments: two\n",
]

combine = [
  "Observation: a magic egg with one stripe and one spot, touching three segments\nResulting segment: two\n",
  "Observation: a magic egg with one stripe and two spots, touching three segments\nResulting segments: one\n",
  "Observation: a magic egg with one stripe and three spots, touching three segments\nResulting segments: none\n",
]

gen_tasks = [
  "Observation: a magic egg with no stripe and no spot, touching three segments\nResulting segments:",
  "Observation: a magic egg with no stripe and two spots, touching one segment\nResulting segments:",
  "Observation: a magic egg with two stripes and one spot, touching three segments\nResulting segments:",
  "Observation: a magic egg with two stripes and three spots, touching three segments\nResulting segments:",
  "Observation: a magic egg with three stripes and two spots, touching three segments\nResulting segments:",
  "Observation: a magic egg with three stripes and four spots, touching two segments\nResulting segments:",
  "Observation: a magic egg with four stripes and no spot, touching three segments\nResulting segments:",
  "Observation: a magic egg with four stripes and four spots, touching four segments\nResulting segments:",
]


# Feature CounterBalancing stimuli
construct_fcb = [
  "Observation: a magic egg with one spot, touching three segments\nResulting segments: three\n",
  "Observation: a magic egg with two spots, touching two segments\nResulting segments: four\n",
  "Observation: a magic egg with three spots, touching one segment\nResulting segments: three\n",
]

decon_fcb = [
  "Observation: a magic egg with one spot and three stripes, touching three segments\nResulting segments: none\n",
  "Observation: a magic egg with two spots and two stripes, touching two segments\nResulting segments: two\n",
  "Observation: a magic egg with three spots and one stripe, touching one segment\nResulting segments: two\n",
]

combine_fcb = [
  "Observation: a magic egg with one spot and one stripe, touching three segments\nResulting segment: two\n",
  "Observation: a magic egg with one spot and two stripes, touching three segments\nResulting segments: one\n",
  "Observation: a magic egg with one spot and three stripes, touching three segments\nResulting segments: none\n",
]

gen_tasks_fcb = [
  "Observation: a magic egg with no spot and no stripe, touching three segments\nResulting segments:",
  "Observation: a magic egg with no spot and two stripes, touching one segment\nResulting segments:",
  "Observation: a magic egg with two spots and one stripe, touching three segments\nResulting segments:",
  "Observation: a magic egg with two spots and three stripes, touching three segments\nResulting segments:",
  "Observation: a magic egg with three spots and two stripes, touching three segments\nResulting segments:",
  "Observation: a magic egg with three spots and four stripes, touching two segments\nResulting segments:",
  "Observation: a magic egg with four spots and no stripe, touching three segments\nResulting segments:",
  "Observation: a magic egg with four spots and four stripes, touching four segments\nResulting segments:",
]


self_report_tasks = {
  
  'exp1-construct-phase1': construct,
  'exp1-construct-phase2': construct + decon,
  
  'exp1-decon-phase1': decon,
  'exp1-decon-phase2': decon + construct,
  
  'exp1-combine-phase1': construct,
  'exp1-combine-phase2': construct + combine,
  
  
  
  'exp2-construct-phase1': construct_fcb,
  'exp2-construct-phase2': construct_fcb + decon_fcb,
  
  'exp2-decon-phase1': decon_fcb,
  'exp2-decon-phase2': decon_fcb + construct_fcb,
  
  'exp2-combine-phase1': construct_fcb,
  'exp2-combine-phase2': construct_fcb + combine_fcb,
  
  
  
  'exp3-combine-phase1': construct,
  'exp3-combine-phase2': construct + combine,
  
  'exp3-flip-phase1': combine,
  'exp3-flip-phase2': combine + construct,
  
  
  
  'exp4-combine-phase1': construct_fcb,
  'exp4-combine-phase2': construct_fcb + combine_fcb,
  
  'exp4-flip-phase1': combine_fcb,
  'exp4-flip-phase2': combine_fcb + construct_fcb,
  
}


## Retrieve OpenAI text completion results

Taken from: <https://wandb.ai/ivangoncharov/GPT-3%20in%20Python/reports/Use-GPT-3-in-Python-with-the-OpenAI-API-and-W-B-Tables--VmlldzoxOTg4NTMz>

Official documentation: <https://beta.openai.com/docs/api-reference>

In [126]:
def get_response(prompt_text, n=1):
  response = openai.Completion.create(
    engine="text-davinci-002",
    prompt=prompt_text,
    temperature=0.5,
    max_tokens=256,
    logprobs=n,
    frequency_penalty=0.0,
    presence_penalty=0.0,
  )
  if n == 1:
    return response['choices'][0]['text']
  else:
    return response['choices'][0]['logprobs']['top_logprobs'][0]

## Get self-reports

In [40]:
sample_reports = {'exp_id':[], 'condition':[], 'phase':[], 'report':[]}

for cond, obs in self_report_tasks.items():
  exp_id = int(cond.split('-')[0][-1])
  condition = cond.split('-')[1]
  phase = cond.split('-')[2][-1]
  
  obs_prompt = ("\n".join(obs) + '\n' + self_report_prompt)
  report = get_response(obs_prompt).strip()
  
  sample_reports['exp_id'].append(exp_id)
  sample_reports['condition'].append(condition)
  sample_reports['phase'].append(phase)
  sample_reports['report'].append(report)


pred_df = pd.DataFrame(sample_reports)
pred_df.to_csv('gpt3-reports.csv')

## Get generalization predictions

In [125]:
# Get top 5 predictions
TOP_N = 5

#gen_preds = pd.DataFrame({'exp_id':[], 'condition':[], 'phase':[], 'task':[], 'prediction':[], 'prob':[]})
for cond, obs in self_report_tasks.items():
  
  gen_preds = pd.read_csv('gpt3-predictions.csv', index_col=0)
  this_gen = {'exp_id':[], 'condition':[], 'phase':[], 'task':[], 'prediction':[], 'prob':[]}
  
  exp_id = int(cond.split('-')[0][-1])
  condition = cond.split('-')[1]
  phase = cond.split('-')[2][-1]
  
  if exp_id % 2 == 1:
    task_set = gen_tasks
  else:
    task_set = gen_tasks_fcb
    
  for i in range(len(task_set)):
    gen_prompt = ("\n".join(obs) + '\n' + task_set[i])
    preds = get_response(gen_prompt, TOP_N)
#     preds = {
#       " four": -3.6177382,
#       " nine": -3.617539,
#       " one": -1.9421821,
#       " six": -2.2962584,
#       " three": -0.4198423
#     }
    
    this_gen['exp_id'] += [exp_id]*TOP_N
    this_gen['condition'] += [condition]*TOP_N
    this_gen['phase'] += [phase]*TOP_N
    this_gen['task'] += [i+1]*TOP_N
    this_gen['prediction'] += [ x.strip() for x in list(preds.keys()) ]
    this_gen['prob'] += [ math.exp(x) for x in list(preds.values()) ]

  
  # Save results   
  this_gen_df = pd.DataFrame(this_gen)
  gen_preds = pd.concat([gen_preds, this_gen_df], ignore_index=True)
  gen_preds.to_csv('gpt3-predictions.csv')
  
  time.sleep(15) # free account can't query all the time :/
  

In [2]:
# Convert to numberical
preds = pd.read_csv('./gpt3-predictions.csv')

,Unnamed: 0,exp_id,condition,phase,task,prediction,prob
0,0,1.0,construct,1,1.0,three,0.695165
1,1,1.0,construct,1,1.0,four,0.017770
2,2,1.0,construct,1,1.0,six,0.075927
3,3,1.0,construct,1,1.0,nine,0.033074
4,4,1.0,construct,1,1.0,one,0.110472
...,...,...,...,...,...,...,...
795,795,4.0,flip,2,8.0,four,0.578766
796,796,4.0,flip,2,8.0,eight,0.019388
797,797,4.0,flip,2,8.0,none,0.241725
798,798,4.0,flip,2,8.0,one,0.091746


In [15]:
w2n.word_to_num('zero')

0

In [9]:
set(preds['prediction'].tolist())

{'all',
 'eight',
 'five',
 'four',
 'nine',
 'none',
 'one',
 'seven',
 'six',
 'three',
 'twelve',
 'two',
 'zero'}

In [22]:
filtered_preds = preds[~preds['prediction'].isin(['none', 'all'])]
#set(filtered_preds['prediction'].to_list())


filtered_preds['num'] = filtered_preds['prediction'].map(lambda x: w2n.word_to_num(x))
filtered_preds.to_csv('./converted.csv')

<ipython-input-22-45c615b47152>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_preds['num'] = filtered_preds['prediction'].map(lambda x: w2n.word_to_num(x))
